# Imports

In [ ]:
import requests
import getpass
import pickle
import io
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Login
https://www.statistikdaten.bayern.de/genesis/online?Menu=Anmeldung#abreadcrumb

In [ ]:
username = input()

In [ ]:
password = getpass.getpass()

## Test login

In [ ]:
class GenesisApi:
    
    def __init__(self, username, password, polling_rate=5):
        self.username = username
        self.password = password
        self.polling_rate = polling_rate
        
        self.__base_url = 'https://www.statistikdaten.bayern.de/genesisWS/rest/2020/'
        
        self.__base_params = {
            'username': username,
            'password': password,
            'language': 'de'
        }
        
        self.__default_table_params = self.__base_params.copy()
        self.__default_table_params.update({
            'name': '',
            'area': 'all',
            'compress': 'false',
            'transpose': 'false',
            'startyear': '',
            'endyear': '',
            'timeslices': '',
            'regionalvariable': '',
            'regionalkey': '',
            'classifyingkey1': '',
            'classifyingvariable2': '',
            'classifyingkey2': '',
            'classifyingvariable3': '',
            'classifyingkey3': '',
            'job': 'true'
        })
        
        self.__default_jobs_params = self.__base_params.copy()
        self.__default_jobs_params.update({
            'selection': '',
            'searchcriterion': 'code',
            'sortcriterion': 'code',
            'type': 'all',
            'area': 'all',
            'pagelength': '100'
        })
        
        self.__default_result_params = self.__base_params.copy()
        self.__default_result_params.update({
            'name': '',
            'area': 'all',
            'compress': 'false'
        })
        
    def check_login(self):
        response = requests.get(self.__base_url + 'helloworld/logincheck', params=self.__base_params)
        b'{"Status":"Sie wurden erfolgreich an- und abgemeldet!","Username":"GB3U65P838"}'
        try:
            return response.json()['Status'] == 'Sie wurden erfolgreich an- und abgemeldet!'
        except Exception as e:
            return False


    def get_table(self, name, startyear=''):
        startyear = str(startyear)
        
        params = self.__default_table_params.copy()
        params['name'] = name
        params['startyear'] = startyear
        
        response = requests.get(self.__base_url + 'data/table', params=params)
        
        data = response.json()
        code = data['Status']['Code']
        if (code == 0): # Success
            return data
        elif (code == 99): # Table is too big a job has been created
            print('Table is too big, created a job.')
            result_name = data['Status']['Content'].split(':', 1)[1][1:]
            return self.get_job_result(result_name)
        else:
            params['password'] = '***'
            print('Error requesting ' + name + ' with params:', params, 'response:', data)
            return data
    
    def is_job_ready(self, name):
        params = self.__default_jobs_params.copy()
        params['selection'] = 'Werteabruf ' + name
        
        response = requests.get(self.__base_url + 'catalogue/jobs', params=params)
        try:
            return response.json()['List'][0]['State'] == 'Fertig'
        except Exception as e:
            return False
    
    def delete_job_result(self, name):
        params = self.__default_result_params.copy()
        params['name'] = name
        response = requests.get(self.__base_url + 'profile/removeResult', params=params)
        return response        
    
    def get_job_result(self, name):
        params = self.__default_result_params.copy()
        params['name'] = name
        
        while(not self.is_job_ready(name)):
            print('Data is not ready waiting ' + str(self.polling_rate) + ' seconds longer.')
            time.sleep(self.polling_rate)
        
        response = requests.get(self.__base_url + 'data/result', params=params)
        self.delete_job_result(name)
        return response.json()


In [ ]:
genesis = GenesisApi(username, password)
genesis.check_login()

# Download data

Note: This takes a long time

In [ ]:
responses_demographic = {}

for year in range(1980, 2020 + 1):
    print('Requesting table for the year ' + str(year))
    response = genesis.get_table('12411-003r', year)
    print('Got data')
    responses_demographic[str(year)] = response

In [ ]:
responses_area = {}

# 33111-201r 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2009, 2010, 2011, 2012, 2013
# 33111-101r 2011 - 2015
# 33111-001r 2014 - 2020

for year in [1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2009, 2010, 2011, 2012, 2013]:
    print('Requesting table for the year ' + str(year))
    response = genesis.get_table('33111-201r', year)
    print('Got data')
    responses_area[str(year)] = response

for year in range(2014, 2020 + 1):
    print('Requesting table for the year ' + str(year))
    response = genesis.get_table('33111-001r', year)
    print('Got data')
    responses_area[str(year)] = response

# Convert to DataFrame

In [ ]:
def convert_to_dataframe(response, start_at_line, date_line, header_line):
    raw_content = response['Object']['Content']
    content = raw_content.split('\n', start_at_line)
    date = content[date_line].split(';',1)[0]
    csv = io.StringIO(content[header_line] + '\n' + content[start_at_line].split('\n__________', 1)[0])
    df = pd.read_csv(csv, ';')
    df['date'] = pd.to_datetime(date, format='%d.%m.%Y')
    return df

## Demographic

In [ ]:
dfs = list()
for year, response in responses_demographic.items():
    df = convert_to_dataframe(response, start_at_line=6, date_line=4, header_line=5)
    dfs.append(df)

df_demographic = pd.concat(dfs, axis=0, ignore_index=True)

column_names = df_demographic.columns.values
column_names[0] = 'AGS'
column_names[1] = 'Gemeinde'
df_demographic.columns = column_names

df_demographic['Gemeinde'] = df_demographic['Gemeinde'].str.strip()
df_demographic['Insgesamt'] = pd.to_numeric(df_demographic['Insgesamt'], errors='coerce')
df_demographic['männlich'] = pd.to_numeric(df_demographic['männlich'], errors='coerce')
df_demographic['weiblich'] = pd.to_numeric(df_demographic['weiblich'], errors='coerce')

In [ ]:
df_demographic
# TODO Filter regierungsbezirke
# TODO Filter male and female

## Area

In [ ]:
dfs = list()
for year, response in responses_area.items():
    df = convert_to_dataframe(response, start_at_line=10, date_line=5, header_line=8)

    column_names = df.columns.values
    column_names[0] = 'AGS'
    column_names[1] = 'Gemeinde'
    df.columns = column_names

    for column_name in column_names[2: len(column_names) - 1]:
        df[column_name] = pd.to_numeric(df[column_name].str.replace(',', '.'), errors='coerce')

    df['Gemeinde'] = df['Gemeinde'].str.strip()
    
    dfs.append(df)

df_area = pd.concat(dfs, axis=0, ignore_index=True)

In [ ]:
df_area

In [ ]:
# TODO Map old area codes to new ones
# TODO Map area codes to sealed and non-sealed
# TODO Filter regierungsbezirke

## Combined

In [ ]:
df_all = pd.merge(df_area, df_demographic,  how='left', on=['AGS', 'Gemeinde', 'date'])
df_all.rename(columns={'Insgesamt_x':'Insgesamt Fläche', 'Insgesamt_y':'Insgesamt Bewohner'}, inplace=True)
df_all

# Save and load data

In [ ]:
df_demographic.to_pickle('df_demographic.pickle')
df_area.to_pickle('df_area.pickle')

with open('responses_demographic.pickle', 'wb') as f:
    pickle.dump(responses_demographic, f, pickle.HIGHEST_PROTOCOL)

with open('responses_area.pickle', 'wb') as f:
    pickle.dump(responses_area, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
df_demographic = pd.read_pickle('df_demographic.pickle')
df_area = pd.read_pickle('df_area.pickle')

with open('responses_demographic.pickle', 'rb') as f:
    responses_demographic = pickle.load(f)
    
with open('responses_area.pickle', 'rb') as f:
    responses_area = pickle.load(f)

# Basic graphs

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
ax.set(yscale="log")
g = sns.lineplot(data=df_demographic[(df_demographic['Gemeinde']=='Friedberg, St') | (df_demographic['Gemeinde']=='Augsburg (Krfr.St)') | (df_demographic['Gemeinde']=='Garmisch-Partenkirchen, M')], style='Gemeinde', x='date', y='Insgesamt', ax=ax)
g.set_title('Einwohner')
g.set(ylim=(1, None))
g
#sns.lineplot(data=df_demographic, style='Gemeinde', x='date', y='Insgesamt', ax=ax)#, ylim=(0,300000))

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
#ax.set(yscale="log")
g = sns.lineplot(data=df_area[(df_area['Gemeinde']=='Friedberg, St') | (df_area['Gemeinde']=='Augsburg (Krfr.St)') | (df_area['Gemeinde']=='Garmisch-Partenkirchen, M')], style='Gemeinde', x='date', y='Waldfläche', ax=ax)
g.set_title('Einwohner')
#g.set(ylim=(0, None))
g

In [ ]:
sealed = [
    'Wohnen',
    'Gewerbe, Industrie',
    'Straße, Weg, Platz',
    '11000 Wohnbaufläche',
    '12100 Industrie und Gewerbe',
    '12200 Handel und Dienstleistung',
    '12300 Versorgungsanlage',
    '12400 Entsorgung',
    '13000 Halde',
    '14000 Bergbaubetrieb',
    '21000 Straßenverkehr',
    '22000 Weg',
    '26000 Schiffsverkehr',
]

unsealed = [
    '15000 Tagebau, Grube, Steinbruch',
    'Abbauland',
    'Grünanlage',
    'Moor',
    'Heide',
    'Landwirtschaftsfläche (ohne Moor, Heide)',
    'Waldfläche',
    'Friedhof',
    'Unland',
    '18400 Grünanlage',
    '19000 Friedhof',
    '31100 Ackerland',
    '31200 Grünland',
    '31300 Gartenland',
    '31400 Weingarten',
    '31500 Obstplantage',
    '32000 Wald',
    '33000 Gehölz',
    '34000 Heide',
    '35000 Moor',
    '36000 Sumpf',
    '37000 Unland, Vegetationslose Fläche',
    
]

undefined = [
    'sonstige Gebäude- und Freifläche',
    'Betriebsfläche (ohne Abbauland)',
    'sonstige Verkehrsfläche',
    '16000 Fläche gemischter Nutzung',
    '17000 Fläche besonderer funktionaler Prägung',
    '18100 Sportanlage',
    'sonstige Erholungsfläche',
    '18200 Freizeitanlage',
    '18300 Erholungsfläche',
    '23000 Platz',
    '24000 Bahnverkehr',
    '25000 Flugverkehr',
    'Flächen anderer Nutzung (ohne Unland, Friedhof)',
    
]

remove = [
    '41000 Fließgewässer',
    '42000 Hafenbecken',
    '43000 Stehendes Gewässer',  
    'Wasserfläche', 
]

In [ ]:
df_area_2 = df_area.copy()

df_area_2['sealed'] = df_area_2.loc[:,sealed].sum(axis=1)
df_area_2['unsealed'] = df_area_2.loc[:,unsealed].sum(axis=1)
df_area_2['remove'] = df_area_2.loc[:,remove].sum(axis=1)
df_area_2['undefined'] = df_area_2.loc[:,undefined].sum(axis=1)

df_area_2.drop(sealed, axis=1, inplace=True)
df_area_2.drop(unsealed, axis=1, inplace=True)
df_area_2.drop(remove, axis=1, inplace=True)
df_area_2.drop(undefined, axis=1, inplace=True)

df_area_2['sealed'] = df_area_2['sealed'] / df_area_2['Insgesamt']
df_area_2['unsealed'] = df_area_2['unsealed'] / df_area_2['Insgesamt']
df_area_2['remove'] = df_area_2['remove'] / df_area_2['Insgesamt']
df_area_2['undefined'] = df_area_2['undefined'] / df_area_2['Insgesamt']

In [ ]:
gem = ['Bayern', 'Oberbayern', 'Schwaben']
f, axs = plt.subplots(1, 3, figsize=(18, 6))

df_area_2 = df_area_2[df_area_2['date'] > pd.to_datetime("1.1.2010", format='%d.%m.%Y')]

for i in range(0, 3):
    g = df_area_2[(df_area_2['Gemeinde']==gem[i])].plot.area(x='date', y=['sealed', 'unsealed', 'remove', 'undefined'], stacked=True, ax=axs[i])
    g.set_title('Flächen in ' + gem[i])
    g.set(ylim=(0, None))

plt.savefig('flächen.svg')

In [ ]:
list(df_area_2['Gemeinde'].unique())